In [1]:
import os
import torch
import clip
import numpy as np
import pandas as pd
import pyarrow.feather as feather
from PIL import Image
from urllib import request
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import torchvision, torch
from sklearn.linear_model import LogisticRegression
import sklearn
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
import time
# proxy="http://sysproxy.wal-mart.com:8080"
# os.environ['http_proxy'] = proxy 
# os.environ['HTTP_PROXY'] = proxy
# os.environ['https_proxy'] = proxy
# os.environ['HTTPS_PROXY'] = proxy


# export HTTP_PROXY=http://sysproxy.wal-mart.com:8080
# export HTTPS_PROXY=http://sysproxy.wal-mart.com:8080

In [4]:
from pprintpp import pprint
gtin_mapping=pd.read_csv(os.path.join(os.getcwd(),"dvc-manual/gtin_attr.csv"))
cols= gtin_mapping.columns


Index(['gtin', 'Acceptable Temperature Range High',
       'Acceptable Temperature Range UOM', 'Added Sugar Per Serving',
       'Is Aerosol', 'Food Allergen Statements', 'alternate_shelves',
       'Assembled Product Height', 'Assembled Product Length',
       'Assembled Product Weight',
       ...
       'Maximum Motor Speed', 'Maximum Temperature', 'Receiver Compatibility',
       'Reserve Capacity', 'SAE DOT-Compliant', 'Shackle Clearance',
       'Shackle Diameter', 'Shackle Length', 'Shank Length', 'Shear Strength'],
      dtype='object', length=1415)


In [18]:
cols= gtin_mapping.columns
for col in cols:
    if 'container' in col.lower():
        print(col)
    
# [col for cols if 'shape' is in col]

Container Type
Has Fuel Container
Servings Per Container
Container Material
Rigid Plastic Packaging Container Indicator
360 Image Container


In [29]:
print(gtin_mapping.shape[0])
col="Container Type"
categories=gtin_mapping.loc[gtin_mapping[col].notna(),["gtin", "Product Name" ,"Container Type","Retail Packaging","Shape","Gemstone Shape", "Pearl Shape"]][col].value_counts().reset_index()
categories.to_csv("shape_categories.csv")

1522


In [3]:
import pandas as pd
import re

# import pandas as pd
# gtin_mapping=pd.read_csv(os.path.join(os.getcwd(),"dvc-manual/520_gtin_product_name.csv"))
# [gtin_mapping[gtin_mapping["gtin"]==gtin_mapping["gtin"].value_counts().index[1]]["product_name"].iloc[i] for i in range(0,4)]

def cleanhtml(raw_html):
    CLEANR = re.compile('<.*?>') # remove html tags
    cleantext = re.sub(CLEANR, '', raw_html)
    pattern= r"\d*\.\d+"# r'[0-9]' # remove decimal numbers
    cleantext=re.sub(pattern, "",cleantext)
    pattern= r'[0-9]' # remove any digits
    cleantext=re.sub(pattern, "",cleantext)
    return cleantext

def get_mapping():
    
    gtin_mapping=pd.read_csv(os.path.join(os.getcwd(),"dvc-manual/gtin_attr.csv"))
    desc_columns = [col for col in gtin_mapping.columns if 'desc' in col.lower() or "name" in col.lower() or "date" in col.lower()]
    desc_columns= ["gtin","KARF Picker Description","Product Long Description","Short Description","Product Name" ]
    gtin_mapping= gtin_mapping.loc[:,desc_columns]
    
    
    gtin_mapping["Product Long Description"]=gtin_mapping.apply(lambda x: cleanhtml(str(x["Product Long Description"])), axis=1)
    gtin_mapping["Short Description"]=gtin_mapping.apply(lambda x: cleanhtml(str(x["Short Description"])), axis=1)
    gtin_mapping["Product Name"]=gtin_mapping.apply(lambda x: cleanhtml(str(x["Product Name"])), axis=1)
    gtin_mapping=gtin_mapping.rename(columns={"Product Long Description": "desc_long", "Short Description": "desc_short", "Product Name": "name","KARF Picker Description":"desc_karf" })

    gtin_mapping["desc_long"]=gtin_mapping.apply(lambda x: str(x["desc_long"]).replace("|", ""),axis=1)
    gtin_mapping["desc_short"]=gtin_mapping.apply(lambda x: str(x["desc_short"]).replace("|", ""),axis=1)
    gtin_mapping["desc_karf"]=gtin_mapping.apply(lambda x: str(x["desc_karf"]).replace("|", ""),axis=1)
    gtin_mapping["name"]=gtin_mapping.apply(lambda x: str(x["name"]).replace("|", ""),axis=1)

    gtin_mapping["gtin"]=gtin_mapping["gtin"].apply(lambda x: str(x).zfill(14))
    
    return gtin_mapping

def create_dataframe(path,label_col):
    loader= torchvision.datasets.ImageFolder(root=path)
    
    df= pd.DataFrame(loader.imgs, columns= ["img_path", "label"])
    df["gtin"]= df.apply(lambda x: x["img_path"].split("/")[-2],axis=1)
    df["gtin"]=df["gtin"].apply(lambda x: str(x).zfill(14))
    
    gtin_mapping= get_mapping()
    
    df= df.merge(gtin_mapping, left_on=['gtin'], right_on=["gtin"], how = "left")
    no_desc_gtin= df[df['name'].isna()].gtin.unique()
    print(f"Gtin's with no description: {no_desc_gtin}")
    print(f"Shape before removing gtin's{df.shape[0]}")
    df= df[~df["gtin"].isin(no_desc_gtin)]
    print(f"Shape after removing gtin's{df.shape[0]}")
    df = df[df[label_col].notna()]
    print(f"Shape after removing na from product name column {df.shape[0]}")
    key_list=[k for k in range(df.shape[0])]
    df["key"]= key_list
    return df

label_col= "name"
test_path= os.path.join(os.getcwd(),'dvc-manual/gtin_60/data/prep/80gtin_allsubfolders/splitfolders/test')
test_df=create_dataframe(test_path, label_col)

train_path= os.path.join(os.getcwd(),'dvc-manual/gtin_60/data/prep/80gtin_allsubfolders/splitfolders/train')
train_df=create_dataframe(train_path, label_col)

val_path= os.path.join(os.getcwd(),'dvc-manual/gtin_60/data/prep/80gtin_allsubfolders/splitfolders/val')
val_df=create_dataframe(val_path, label_col)

## remove gtin's with no description available

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:46: DtypeWarning: Columns (0,27,49,73,144,166,182,193,231,243,270,276,327,328,330,345,390,393,400,402,404,406,423,426,427,438,445,446,452,459,460,466,469,483,488,489,490,491,493,494,498,500,501,502,503,505,507,509,510,511,513,519,528,534,537,546,575,580,586,591,594,605,614,619,621,627,629,630,639,640,641,642,660,661,663,666,667,672,676,684,685,687,689,694,697,699,700,701,705,706,707,710,712,713,715,717,719,724,726,730,732,733,736,737,740,741,743,744,745,746,748,750,752,753,755,756,760,761,762,763,765,767,770,771,772,778,780,781,782,783,785,788,793,794,796,797,798,799,800,804,805,806,807,808,809,811,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,832,833,834,835,836,837,838,839,840,841,842,844,845,846,848,849,850,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,868,869,870,871,872,874,877,881,882,885,889,891,892,907,908,911,916,917,923,924,931,933,934,949,951,957,958,965,967,968,970,973,974,978,984,9

Gtin's with no description: ['00016000124790' '00044700360019' '00703820010944']
Shape before removing gtin's10661
Shape after removing gtin's10132
Shape after removing na from product name column 10132
Gtin's with no description: ['00016000124790' '00044700360019' '00703820010944']
Shape before removing gtin's34401
Shape after removing gtin's32691
Shape after removing na from product name column 32691
Gtin's with no description: ['00016000124790' '00044700360019' '00703820010944']
Shape before removing gtin's7914
Shape after removing gtin's7520
Shape after removing na from product name column 7520


In [69]:

# # batch_num=0
# # for batch in train_dataloader:
# #     batch_num+=1
# #     if batch_num==52:
# #         print(batch[1])
# from transformers import CLIPTokenizer, CLIPTextModel
# text_model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
# tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")


# vision_model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
from random import randint, choice
descriptions = train_dataloader.dataset.caption[0].split('\n')
descriptions = list(filter(lambda t: len(t) > 0, descriptions))
description = choice(descriptions)
description

'Nutella Chocolate Hazelnut Spread Perfect on Pancakes  oz Jar'

In [77]:
key_list=[k for k in range(train_df.shape[0])]
train_df["key"]= key_list
train_df.head()

,img_path,label,gtin,desc_karf,desc_long,desc_short,name,key
0,/home/jupyter/dvc-manual/gtin_60/data/prep/80g...,0,00009800800254,Nutella 33.5 oz,One oz jar of delicious Nutella hazelnut spre...,The great taste of Nutella makes weekend break...,Nutella Chocolate Hazelnut Spread Perfect on P...,0
1,/home/jupyter/dvc-manual/gtin_60/data/prep/80g...,0,00009800800254,Nutella 33.5 oz,One oz jar of delicious Nutella hazelnut spre...,The great taste of Nutella makes weekend break...,Nutella Chocolate Hazelnut Spread Perfect on P...,1
2,/home/jupyter/dvc-manual/gtin_60/data/prep/80g...,0,00009800800254,Nutella 33.5 oz,One oz jar of delicious Nutella hazelnut spre...,The great taste of Nutella makes weekend break...,Nutella Chocolate Hazelnut Spread Perfect on P...,2
3,/home/jupyter/dvc-manual/gtin_60/data/prep/80g...,0,00009800800254,Nutella 33.5 oz,One oz jar of delicious Nutella hazelnut spre...,The great taste of Nutella makes weekend break...,Nutella Chocolate Hazelnut Spread Perfect on P...,3
4,/home/jupyter/dvc-manual/gtin_60/data/prep/80g...,0,00009800800254,Nutella 33.5 oz,One oz jar of delicious Nutella hazelnut spre...,The great taste of Nutella makes weekend break...,Nutella Chocolate Hazelnut Spread Perfect on P...,4


### Requirements
- CLIP encoder, CLIP text encoder, CLIP tokenizer, CLIP processor

1. Prepare data into the required format
2. Load text encoder and image encoder
    Image processor CLIP preprocess
    CLIP tokenizer
        Default: clip.tokenize
        Huggingface: 
        
root_folder
    img1.png
    img1.txt
    
    
    

In [74]:
from PIL import Image

BATCH_SIZE=64

# class image_caption_dataset(Dataset):
#     def __init__(self, df):

#         self.images = df["img_path"].tolist()
#         self.caption = df["name"].tolist()
        
#     def __len__(self):
#         return len(self.caption)

#     def __getitem__(self, idx):
#         images = preprocess(Image.open(self.images[idx])) #preprocess from clip.load
#         caption = self.caption[idx]
#         return images,caption


# class image_caption_dataset(Dataset):
#     def __init__(self, df):

#         self.images = df["img_path"].tolist()
#         self.caption = df["name"].tolist()
#         self.device = (
#             "cuda:0" if torch.cuda.is_available() else "cpu"
#         )  # If using GPU then use mixed precision training.
#         _, self.preprocess = clip.load(
#             "ViT-B/32", device=self.device, jit=False
#         )  # Must set jit=False for training

#     def __len__(self):
#         return len(self.caption)

#     def __getitem__(self, idx):
#         images = self.preprocess(Image.open(self.images[idx]))  # preprocess from clip.load
#         # caption=torch.cat([clip.tokenize(c) for c in self.caption])
#         caption=clip.tokenize(self.caption[idx])[0]
#         # caption = self.caption[idx]
#         return images, caption

class image_caption_dataset(Dataset):
    def __init__(self, df):

        self.images = df["img_path"].tolist()
        self.caption = df["name"].tolist()
        self.keys= df["key"].tolist()
        self.image_transform = T.Compose([
            T.Lambda(self.fix_img),
            T.RandomResizedCrop(image_size,
                            scale=(self.resize_ratio, 1.),
                            ratio=(1., 1.)),
            T.ToTensor(),
            T.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
            ])
        # self.tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
        # self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        self.device = (
            "cuda:0" if torch.cuda.is_available() else "cpu"
        )  # If using GPU then use mixed precision training.
        _, self.preprocess = clip.load(
            "ViT-B/32", device=self.device, jit=False
        )  # Must set jit=False for training

    def __len__(self):
        return len(self.caption)

    def __getitem__(self, idx):
        
        images = self.preprocess(Image.open(self.images[idx]))
        caption=clip.tokenize(self.caption[idx])[0]
        
        # caption = self.caption[idx]
        # caption= self.tokenizer(self.caption[idx])[0]
        
        return images, caption


dataset = image_caption_dataset(train_df.loc[:,["img_path","name"]])
train_dataloader = DataLoader(dataset,batch_size = BATCH_SIZE) #Define your own dataloader

dataset = image_caption_dataset(val_df.loc[:,["img_path","name"]])
val_loader = DataLoader(dataset,batch_size = BATCH_SIZE) #Define your own dataloader

dataset = image_caption_dataset(test_df.loc[:,["img_path","name"]])
test_loader = DataLoader(dataset,batch_size = BATCH_SIZE) #Define your own dataloader



In [99]:
# tokenizer(train_df["name"].iloc[0:63])
text=tokenizer([row[1] for row in train_df["name"].iloc[0:64]], padding=True, truncation=True, return_tensors="pt")
text_mbs_ids = torch.chunk(torch.arange(64), 1)

text_mbs = []
for s in text_mbs_ids:
    d = {}
    for key in list(text.keys()):
        d[key] = text[key][s]
    text_mbs.append(d)
    

In [115]:
# {k: v for k, v in zip(train_df.loc[:,["key"]],train_df.loc[:,["img_path"]])}

In [90]:
# tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = AutoTokenizer.from_pretrained("johngiorgi/declutr-sci-base")
for batch in train_dataloader:
    print(tokenizer(batch[1][1]))
    #print(tokenizer(row[1] for row in batch, padding=True, truncation=True, return_tensors="pt"))
    break

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [78]:
# print(train_dataloader.batch_size)
# n = math.ceil(train_dataloader.batch_size// 64)
# text_mbs_ids = torch.chunk(torch.arange(64), 32)

# for s in text_mbs_ids:
#     d = {}
#     for key in list(text.keys()):
#         d[key] = text[key][s]
#     text_mbs.append(d)

In [79]:
num_classes=np.unique(np.array(train_dataloader.dataset.caption))

In [41]:
# import math


# for train_batch in train_dataloader:
#     image, text = train_batch
#     print(f"Length of image batch is {image.shape}")
#     print(f"Length of text batch is {text.shape}")
#     n = math.ceil(len(image) // 128)
#     print(n)
#     image_mbs = torch.chunk(image, n)
#     text_mbs = torch.chunk(text, n)

#     # calculate original statistics
#     # with torch.no_grad():
#     #     ims = [F.normalize(self.model.encode_image(im), dim=1) for im in image_mbs]
#     #     txt = [F.normalize(self.model.encode_text(t), dim=1) for t in text_mbs]




In [40]:
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# model, preprocess = clip.load("ViT-B/32",jit=False) #Must set jit=False for training


# # from torchvision.models import resnet50
# # img_encoder = resnet50(pretrained=True)
# # img_encoder.fc = torch.nn.Linear(2048, 768)
# # print(img_encoder.state_dict)

In [70]:
#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 

device = "cuda:0" if torch.cuda.is_available() else "cpu" # If using GPU then use mixed precision training.
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training
if device == "cpu":
    model.float()
else :
    clip.model.convert_weights(model) # Actually this line is unnecessary since clip by default already on float16

loss_img = torch.nn.CrossEntropyLoss()
loss_txt = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2) #Params used from paper, the lr is smaller, more safe for fine tuning to new dataset

EPOCH=150


print("Start Training")
for epoch in range(EPOCH):
    with tqdm(train_dataloader, unit="batch") as tepoch:
        for list_image, list_txt in tepoch:
        # for batch in train_dataloader:
            tepoch.set_description(f"Epoch {epoch}")
            optimizer.zero_grad()
            # batch_num+=1
            # print(f'Batch number = {batch_num}')

            # list_image,list_txt = batch #list_images is list of image in numpy array(np.uint8), or list of PIL images

            images=list_image.to(device)
            # images= torch.stack([preprocess(Image.fromarray(img)) for img in list_image],dim=0).to(device) # omit the Image.fromarray if the images already in PIL format, change this line to images=list_image if using preprocess inside the dataset class
            texts = clip.tokenize(list_txt).to(device)

            logits_per_image, logits_per_text = model(images, texts)

            ground_truth = torch.arange(len(list_image),dtype=torch.long,device=device)

            total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
            acc_i = (torch.argmax(image_logits, 1) == ground_truth).sum()
            acc_t = (torch.argmax(image_logits, 0) == ground_truth).sum()
            total_loss.backward()
            if device == "cpu":
                optimizer.step()
            else : 
                convert_models_to_fp32(model)
                optimizer.step()
                clip.model.convert_weights(model)
            tepoch.set_postfix(loss=total_loss.item())
            time.sleep(0.1)
            

Start Training


Epoch 0: 100%|█████████▉| 510/511 [10:28<00:01,  1.23s/batch, loss=4.16]


ValueError: Expected input batch_size (51) to match target batch_size (64).

In [126]:
torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': total_loss,
        }, f"model_checkpoint/model_10.pt") #just change to your preferred folder/filename

NameError: name 'train_dataloader' is not defined

In [ ]:
class CLIPWrapper(pl.LightningModule):
    def __init__(self,
                 model_name: str,
                 config: dict,
                 minibatch_size: int
                 ):
        """A lightning wrapper for a CLIP model as specified in the paper.
        Args:
            model_name (str): A case sensitive visual model name.
            config (dict): A dictionary containing the CLIP instantiation parameters.
        """
        super().__init__()

        self.model_name = model_name
        self.model = CLIP(**config)
        self.minibatch_size = minibatch_size
        self.isViT = 'ViT' in self.model_name

        self.automatic_optimization = False
    
    # Sourced from https://github.com/PyTorchLightning/pytorch-lightning/issues/5449
    @property
    def num_training_steps(self) -> int:
        """Total training steps inferred from datamodule and devices."""
        dataset = self.train_dataloader()
        if self.trainer.max_steps:
            return self.trainer.max_steps

        dataset_size = len(dataset)

        num_devices = max(1, self.trainer.num_gpus, self.trainer.num_processes)
        if self.trainer.tpu_cores:
            num_devices = max(num_devices, self.trainer.tpu_cores)

        effective_batch_size = dataset.batch_size * self.trainer.accumulate_grad_batches * num_devices
        return (dataset_size // effective_batch_size) * self.trainer.max_epochs

    # Training loss: https://github.com/openai/CLIP/issues/83
    # Mini-batching thanks to https://github.com/crowsonkb / https://twitter.com/RiversHaveWings
    # Multi-GPU support: https://github.com/MicPie/clasp
    def training_step(self, train_batch, idx):
        # get optimizers and scheduler
        optimizer = self.optimizers()

        image, text = train_batch
        n = math.ceil(len(image) // self.minibatch_size)
        image_mbs = torch.chunk(image, n)
        text_mbs = torch.chunk(text, n)

        # calculate original statistics
        with torch.no_grad():
            ims = [F.normalize(self.model.encode_image(im), dim=1) for im in image_mbs]
            txt = [F.normalize(self.model.encode_text(t), dim=1) for t in text_mbs]
            # gather from all GPUs
            ims = self.all_gather(torch.cat(ims))
            txt = self.all_gather(torch.cat(txt))

            if len(ims.shape) == 3:
                ims = list(ims)
                txt = list(txt)
            else:
                ims = [ims]
                txt = [txt]

            image_logits = torch.cat(ims) @ torch.cat(txt).t() * self.model.logit_scale.exp()
            ground_truth = torch.arange(len(image_logits)).type_as(image_logits).long()
            loss = (F.cross_entropy(image_logits, ground_truth) + F.cross_entropy(image_logits.t(), ground_truth)).div(2)
            acc_i = (torch.argmax(image_logits, 1) == ground_truth).sum()
            acc_t = (torch.argmax(image_logits, 0) == ground_truth).sum()
            self.log_dict({'loss': loss / len(ims), 'acc': (acc_i + acc_t) / 2 / len(image) / len(ims)}, prog_bar=True)

        if isinstance(optimizer, list):
            optimizer = optimizer[0]
        optimizer.zero_grad()

        # image loss
        for j, mb in enumerate(image_mbs):
            images_tmp = copy.deepcopy(ims)
            images_tmp[self.global_rank][j*self.minibatch_size:(j+1)*self.minibatch_size] = F.normalize(self.model.encode_image(mb), dim=1)
            image_logits = torch.cat(images_tmp) @ torch.cat(txt).t() * self.model.logit_scale.exp()
            ground_truth = torch.arange(len(image_logits)).type_as(image_logits).long()
            loss = (F.cross_entropy(image_logits, ground_truth) + F.cross_entropy(image_logits.t(), ground_truth))/2
            self.manual_backward(loss)

        # text loss
        for j, mb in enumerate(text_mbs):
            text_tmp = copy.deepcopy(txt)
            text_tmp[self.global_rank][j*self.minibatch_size:(j+1)*self.minibatch_size] = F.normalize(self.model.encode_text(mb), dim=1)
            image_logits = torch.cat(ims) @ torch.cat(text_tmp).t() * self.model.logit_scale.exp()
            loss = (F.cross_entropy(image_logits, ground_truth) + F.cross_entropy(image_logits.t(), ground_truth))/2
            self.manual_backward(loss)

        optimizer.step()
        lr_scheduler = self.lr_schedulers()
        lr_scheduler.step()
        self.model.logit_scale.data.clamp_(-np.log(100), np.log(100))

    def validation_step(self, val_batch, idx):
        image, text = val_batch
        image_logits, text_logits = self.forward(image, text)
        ground_truth = torch.arange(len(image_logits))
        loss = (F.cross_entropy(image_logits, ground_truth) + F.cross_entropy(text_logits, ground_truth)).div(2)
        self.log('val_loss', loss)

    def configure_optimizers(self):
        lr = {
            "RN50": 5e-4,
            "RN101": 5e-4,
            "RN50x4": 5e-4,
            "RN50x16": 4e-4,
            "RN50x64": 3.6e-4,
            "ViT-B/32": 5e-4,
            "ViT-B/16": 5e-4,
            "ViT-L/14": 4e-4,
            "ViT-L/14-336px": 2e-5
        }[self.model_name]

        optimizer = torch.optim.AdamW(
            self.model.parameters(),
            lr=lr,
            betas=(
                0.9,
                0.98 if self.isViT else 0.999
            ),
            eps=1e-6 if self.isViT else 1e-8,
            weight_decay=0.2
        )

        # Source: https://github.com/openai/CLIP/issues/107
        # Use pip install 'git+https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup'
        lr_scheduler = CosineAnnealingWarmupRestarts(
            optimizer,
            first_cycle_steps=self.num_training_steps,
            cycle_mult=1.0,
            max_lr=lr,
            min_lr=0,
            warmup_steps=2000
        )

        return {'optimizer': optimizer, 'lr_scheduler': lr_scheduler}

### Debugging

In [116]:
import pandas as pd
import re
import torchvision
from PIL import Image
import clip
import torch
import os
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPTokenizer, CLIPProcessor
# import pandas as pd
# gtin_mapping=pd.read_csv(os.path.join(os.getcwd(),"dvc-manual/520_gtin_product_name.csv"))
# [gtin_mapping[gtin_mapping["gtin"]==gtin_mapping["gtin"].value_counts().index[1]]["product_name"].iloc[i] for i in range(0,4)]


def cleanhtml(raw_html):
    CLEANR = re.compile("<.*?>")  # remove html tags
    cleantext = re.sub(CLEANR, "", raw_html)
    pattern = r"\d*\.\d+"  # r'[0-9]' # remove decimal numbers
    cleantext = re.sub(pattern, "", cleantext)
    pattern = r"[0-9]"  # remove any digits
    cleantext = re.sub(pattern, "", cleantext)
    return cleantext


def get_mapping():
    gtin_mapping = pd.read_csv("/home/jupyter/dvc-manual/gtin_attr.csv")
    desc_columns = [
        col
        for col in gtin_mapping.columns
        if "desc" in col.lower() or "name" in col.lower() or "date" in col.lower()
    ]
    desc_columns = [
        "gtin",
        "KARF Picker Description",
        "Product Long Description",
        "Short Description",
        "Product Name",
    ]
    gtin_mapping = gtin_mapping.loc[:, desc_columns]
    gtin_mapping["Product Long Description"] = gtin_mapping.apply(
        lambda x: cleanhtml(str(x["Product Long Description"])), axis=1
    )
    gtin_mapping["Short Description"] = gtin_mapping.apply(
        lambda x: cleanhtml(str(x["Short Description"])), axis=1
    )
    gtin_mapping["Product Name"] = gtin_mapping.apply(
        lambda x: cleanhtml(str(x["Product Name"])), axis=1
    )
    gtin_mapping = gtin_mapping.rename(
        columns={
            "Product Long Description": "desc_long",
            "Short Description": "desc_short",
            "Product Name": "name",
            "KARF Picker Description": "desc_karf",
        }
    )
    gtin_mapping["desc_long"] = gtin_mapping.apply(
        lambda x: str(x["desc_long"]).replace("|", ""), axis=1
    )
    gtin_mapping["desc_short"] = gtin_mapping.apply(
        lambda x: str(x["desc_short"]).replace("|", ""), axis=1
    )
    gtin_mapping["desc_karf"] = gtin_mapping.apply(
        lambda x: str(x["desc_karf"]).replace("|", ""), axis=1
    )
    gtin_mapping["name"] = gtin_mapping.apply(
        lambda x: str(x["name"]).replace("|", ""), axis=1
    )
    gtin_mapping["gtin"] = gtin_mapping["gtin"].apply(lambda x: str(x).zfill(14))
    return gtin_mapping


def create_dataframe(path, label_col):
    loader = torchvision.datasets.ImageFolder(root=path)

    df = pd.DataFrame(loader.imgs, columns=["img_path", "label"])
    df["gtin"] = df.apply(lambda x: x["img_path"].split("/")[-2], axis=1)
    df["gtin"] = df["gtin"].apply(lambda x: str(x).zfill(14))

    gtin_mapping = get_mapping()

    df = df.merge(gtin_mapping, left_on=["gtin"], right_on=["gtin"], how="left")
    no_desc_gtin = df[df["name"].isna()].gtin.unique()
    print(f"Gtin's with no description: {no_desc_gtin}")
    print(f"Shape before removing gtin's{df.shape[0]}")
    df = df[~df["gtin"].isin(no_desc_gtin)]
    print(f"Shape after removing gtin's{df.shape[0]}")
    df = df[df[label_col].notna()]
    print(f"Shape after removing na from product name column {df.shape[0]}")
    return df

In [117]:
testing=create_dataframe("/home/jupyter/dvc-manual/gtin_60/data/prep/80gtin_allsubfolders/splitfolders/train","name")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:80: DtypeWarning: Columns (0,27,49,73,144,166,182,193,231,243,270,276,327,328,330,345,390,393,400,402,404,406,423,426,427,438,445,446,452,459,460,466,469,483,488,489,490,491,493,494,498,500,501,502,503,505,507,509,510,511,513,519,528,534,537,546,575,580,586,591,594,605,614,619,621,627,629,630,639,640,641,642,660,661,663,666,667,672,676,684,685,687,689,694,697,699,700,701,705,706,707,710,712,713,715,717,719,724,726,730,732,733,736,737,740,741,743,744,745,746,748,750,752,753,755,756,760,761,762,763,765,767,770,771,772,778,780,781,782,783,785,788,793,794,796,797,798,799,800,804,805,806,807,808,809,811,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,832,833,834,835,836,837,838,839,840,841,842,844,845,846,848,849,850,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,868,869,870,871,872,874,877,881,882,885,889,891,892,907,908,911,916,917,923,924,931,933,934,949,951,957,958,965,967,968,970,973,974,978,984,9

Gtin's with no description: ['00016000124790' '00044700360019' '00703820010944']
Shape before removing gtin's34401
Shape after removing gtin's32691
Shape after removing na from product name column 32691


In [ ]:



class TextImageDataset(Dataset):
    def __init__(self,
                 folder: str,
                 image_size=224,
                 resize_ratio=0.75,
                 shuffle=False,
                 custom_tokenizer=False
                 ):
        """Create a text image dataset from a directory with congruent text and image names.

        Args:
            folder (str): Folder containing images and text files matched by their paths' respective "stem"
            image_size (int, optional): The size of outputted images. Defaults to 224.
            resize_ratio (float, optional): Minimum percentage of image contained by resize. Defaults to 0.75.
            shuffle (bool, optional): Whether or not to have shuffling behavior during sampling. Defaults to False.
            custom_tokenizer (bool, optional): Whether or not there is a custom tokenizer. Defaults to False.
        """
        super().__init__()
        self.shuffle = shuffle

        train_df= create_dataframe(self.folder, "name")
        self.keys = self.keys = list(train_df["key"])
        self.text_files = {k:v for k, v in zip(train_df["key"],train_df["name"])}
        self.image_files = {k:v for k, v in zip(train_df["key"],train_df["img_path"])}
        
        self.resize_ratio = resize_ratio
        self.image_transform = T.Compose([
            T.Lambda(self.fix_img),
            T.RandomResizedCrop(image_size,
                                scale=(self.resize_ratio, 1.),
                                ratio=(1., 1.)),
            T.ToTensor(),
            T.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ])
        self.custom_tokenizer = custom_tokenizer

    def __len__(self):
        return len(self.keys)
    
    def fix_img(self, img):
        return img.convert('RGB') if img.mode != 'RGB' else img

    def random_sample(self):
        return self.__getitem__(randint(0, self.__len__() - 1))

    def sequential_sample(self, ind):
        if ind >= self.__len__() - 1:
            return self.__getitem__(0)
        return self.__getitem__(ind + 1)

    def skip_sample(self, ind):
        if self.shuffle:
            return self.random_sample()
        return self.sequential_sample(ind=ind)

    def __getitem__(self, ind):
        key = self.keys[ind]

        text_file = self.text_files[key]
        image_file = self.image_files[key]

        descriptions = text_file.read_text().split('\n')
        descriptions = list(filter(lambda t: len(t) > 0, descriptions))
        try:
            description = choice(descriptions)
        except IndexError as zero_captions_in_file_ex:
            print(f"An exception occurred trying to load file {text_file}.")
            print(f"Skipping index {ind}")
            return self.skip_sample(ind)

        tokenized_text = description if self.custom_tokenizer else clip.tokenize(description)[0]

        try:
            image_tensor = self.image_transform(PIL.Image.open(image_file))
        except (PIL.UnidentifiedImageError, OSError) as corrupt_image_exceptions:
            print(f"An exception occurred trying to load file {image_file}.")
            print(f"Skipping index {ind}")
            return self.skip_sample(ind)

        # Success
        return image_tensor, tokenized_text

In [119]:
train_dataloader.num_workers

0